In [4]:
# Load pre-existing spacy model
import spacy
nlp=spacy.load('en_core_web_sm')

# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [1]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [2]:
with open('custom_dataset_Cappuccino.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"') for line in file]

In [3]:
train_set = []

for line in lines:
  train_set.append(generate_entity_output(line,"Cappuccino","BEVERAGE"))

In [5]:
from spacy import training
# Adding labels to the `ner`

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [6]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [11]:
train_set[:10]

[('Could I get a small Cappuccino with caramel syrup, a swirl of caramel syrup, a pinch of cinnamon, a drizzle of chocolate sauce, whipped cream, a shot of hazelnut, almond milk, a dusting of cocoa powder, a hint of vanilla, and a touch of honey, please?',
  {'entities': [(20, 29, 'BEVERAGE')]}),
 ('May I have a large Cappuccino with almond milk, a sprinkle of cocoa powder, a dash of cinnamon, whipped cream, and a touch of honey?',
  {'entities': [(19, 28, 'BEVERAGE')]}),
 ('Could I get a medium Cappuccino with a drizzle of chocolate sauce, a shot of hazelnut, almond milk, caramel syrup, a swirl of caramel syrup, a dusting of cocoa powder, whipped cream, a hint of vanilla, a pinch of cinnamon, a dash of vanilla syrup, and a touch of honey, please?',
  {'entities': [(21, 30, 'BEVERAGE')]}),
 ('May I have a large Cappuccino with almond milk, a sprinkle of cocoa powder, a dash of cinnamon, a drizzle of chocolate sauce, a swirl of caramel syrup, whipped cream, and a touch of honey, please?

In [9]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        print("Losses", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "May I have a large Cappuccino with almond milk, a ..." with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I order a small Cappuccino with chocolate spri..." with entities "[(20, 29, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a small Cappuccino with caram

Losses {'ner': 12.738295892012543}
Losses {'ner': 15.511811819007857}
Losses {'ner': 21.48695696081706}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a small Cappuccino with caramel syrup, a ..." with entities "[(17, 26, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a medium Cappuccino with a drizzle of ..." with entities "[(21, 30, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I get a small Cappuccino with a shot 

Losses {'ner': 25.86450800468652}
Losses {'ner': 28.09499493250895}
Losses {'ner': 28.224595379479446}
Losses {'ner': 29.121915967679623}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I order a large Cappuccino with a shot of haze..." with entities "[(20, 29, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a small Cappuccino with a pinch of cinnam..." with entities "[(17, 26, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I get a medium Cappuccino with a swir

...

Losses {'ner': 30.851086619773195}
Losses {'ner': 30.851088140484546}
Losses {'ner': 30.85108850408642}
Losses {'ner': 30.851088522802122}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'll have a small Cappuccino with a shot of hazeln..." with entities "[(18, 27, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a large Cappuccino with almond milk, c..." with entities "[(20, 29, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "May I have a small Cappuccino with cinnam

Losses {'ner': 30.85109363852686}
Losses {'ner': 30.851097776322117}
Losses {'ner': 5.4407403604645135e-09}
Losses {'ner': 3.58728074318956e-08}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'll take a medium Cappuccino with a dusting of nu..." with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a large Cappuccino with a shot of vanilla..." with entities "[(17, 26, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I get a medium Cappuccino to go?" wit

Losses {'ner': 1.248257025284285e-06}
Losses {'ner': 1.2490294154988996e-06}
Losses {'ner': 1.579868322944395e-06}
Losses {'ner': 1.5811933454544532e-06}
Losses {'ner': 1.5819648580729497e-06}
Losses {'ner': 1.5820237220235392e-06}
Losses {'ner': 4.9789633019297456e-06}
Losses {'ner': 4.979436336771174e-06}
Losses {'ner': 5.0890104274653384e-06}
Losses {'ner': 5.771031714348703e-06}
Losses {'ner': 5.771211231693342e-06}
Losses {'ner': 8.672987939781084e-06}
Losses {'ner': 8.673585683906269e-06}
Losses {'ner': 1.3010677963118472e-05}
Losses {'ner': 1.303403814737496e-05}
Losses {'ner': 1.4086370396327701e-05}
Losses {'ner': 1.4087370305050542e-05}
Losses {'ner': 1.4095385690539517e-05}
Losses {'ner': 1.4116281448888545e-05}
Losses {'ner': 1.4275939836785465e-05}
Losses {'ner': 1.4276051118091729e-05}
Losses {'ner': 1.4299653840410826e-05}
Losses {'ner': 1.430108067468717e-05}
Losses {'ner': 3.5922195386078937e-12}
Losses {'ner': 8.647492598085214e-11}
Losses {'ner': 0.000243438811090167

In [10]:
# Testing the model
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities []


# Without Fine tuning
Only adding the entity to the ruler

In [22]:
import spacy

In [23]:
nlp = spacy.load("en_core_web_sm")

In [24]:
ruler = nlp.add_pipe('entity_ruler')

In [25]:
patterns = [
    {"label":"BEVERAGE","pattern":"Cappuccino"}
]

In [26]:
ruler.add_patterns(patterns)

In [27]:
# Testing the model
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'PERSON')]


# Fine Tuning

In [29]:
# Load pre-existing spacy model
import spacy
nlp=spacy.load('en_core_web_sm')


In [30]:
# Adding the entity to the ruler before fine tuning
ruler = nlp.add_pipe('entity_ruler')

patterns = [
    {"label":"BEVERAGE","pattern":"Cappuccino"}
]

ruler.add_patterns(patterns)

In [31]:
# Testing before fine tuning
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'PERSON')]


We can see that it is incorrectly categorizing "Cappuccino" as a "PERSON"

In [33]:
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [34]:
def generate_entity_output(sentence, word, entity_name):
    start_index = sentence.index(word)
    end_index = start_index + len(word) - 1
    output = {
        "entities": [
            (start_index, end_index, entity_name)
        ]
    }
    return (sentence, output)

In [35]:
with open('custom_dataset_Cappuccino.txt', encoding='UTF-8') as file:
    # lines = [line.rstrip() for line in file]
    lines = [line.strip('"').rstrip().rstrip('"') for line in file]

In [36]:
train_set = []

for line in lines:
  train_set.append(generate_entity_output(line,"Cappuccino","BEVERAGE"))

In [37]:
from spacy import training
# Adding labels to the `ner`

for _, annotations in train_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [38]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [39]:
train_set[0]

("I'll have a small Cappuccino, please.", {'entities': [(18, 27, 'BEVERAGE')]})

In [40]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(100):

    # shuufling examples  before every iteration
    random.shuffle(train_set)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_set, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        # Update the model with iterating each text
        for i in range(len(texts)):
            doc = nlp.make_doc(texts[i])
            example.append(Example.from_dict(doc, annotations[i]))
        
        # Update the model
        nlp.update(example, drop=0.5, losses=losses)

        print("Losses", losses)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a large Cappuccino with whipped cream, al..." with entities "[(17, 26, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a large Cappuccino with almond milk, c..." with entities "[(20, 29, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "May I have a large Cappuccino with almond

Losses {'ner': 8.975206826665088}
Losses {'ner': 10.633735810218639}
Losses {'ner': 12.040791142873362}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "May I have a small Cappuccino with caramel syrup, ..." with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a small Cappuccino with a sprinkle of coc..." with entities "[(17, 26, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I get a large Cappuccino with almond 

Losses {'ner': 12.427684418472074}
Losses {'ner': 12.63234068710552}
Losses {'ner': 14.55795508679386}
Losses {'ner': 14.883638094712895}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a large Cappuccino with a swirl of car..." with entities "[(20, 29, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a medium Cappuccino with whipped cream, a..." with entities "[(18, 27, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'll have a large Cappuccino with a swirl

...

Losses {'ner': 14.88838169089262}
Losses {'ner': 14.888387066497907}
Losses {'ner': 14.889773916496482}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'll take a small Cappuccino with a swirl of caram..." with entities "[(18, 27, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Can I get a medium Cappuccino to go?" with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "May I have a small Cappuccino with almond milk and..." wit

Losses {'ner': 14.889774034583}
Losses {'ner': 14.889774655518938}
Losses {'ner': 14.889775335509592}
Losses {'ner': 14.889775429023807}


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "May I have a small Cappuccino with a sprinkle of c..." with entities "[(19, 28, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Could I get a medium Cappuccino with almond milk?" with entities "[(21, 30, 'BEVERAGE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like a large Cappuccino with extra foam."

Losses {'ner': 14.88977975891416}
Losses {'ner': 14.889779836499228}
Losses {'ner': 14.889779881571613}
Losses {'ner': 8.110954110824345e-10}
Losses {'ner': 4.100987286635396e-09}
Losses {'ner': 4.159862043849627e-09}
Losses {'ner': 6.167225325762456e-07}
Losses {'ner': 1.1617431538386511e-06}
Losses {'ner': 1.1629009488375324e-06}
Losses {'ner': 1.3626960417448556e-06}
Losses {'ner': 1.3675495749614932e-06}
Losses {'ner': 9.87858802398661e-06}
Losses {'ner': 9.878627699184847e-06}
Losses {'ner': 9.87862951210177e-06}
Losses {'ner': 9.888841616652408e-06}
Losses {'ner': 9.891001768194043e-06}
Losses {'ner': 9.917153273934562e-06}
Losses {'ner': 9.917186677994653e-06}
Losses {'ner': 9.917188872267505e-06}
Losses {'ner': 9.917188880781294e-06}
Losses {'ner': 9.944791047726085e-06}
Losses {'ner': 9.944791529509818e-06}
Losses {'ner': 9.946788388265148e-06}
Losses {'ner': 9.963729178269287e-06}
Losses {'ner': 9.963746886800917e-06}
Losses {'ner': 9.963748281796449e-06}
Losses {'ner': 9.963

In [41]:
# Testing the model
doc = nlp("I'd like a large Cappuccino with extra foam.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Cappuccino', 'BEVERAGE')]
